# Selecting a Restaurant Location

Now, we need to use the data we obtained to help make a decision on where to locate the business. For this example, we will explore two possible scenarios:

- Casual dining Thai restaurant targeting upper-middle class families and working professionals
- Pub targeting college/university students

A metric needs to be created so each potential venue can be graded on multiple factors - level of competition, size of potential market, and investment required.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import h3
from sklearn.preprocessing import MaxAbsScaler

import plotly.express as px
import folium

In [2]:
df = gpd.read_feather('../data/bangalore_clustered.feather')
df.head()

,id,cluster,geometry,address,pop_density,cost_sqft,ATM,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Quick Bites,Residence,Restaurant,Salon,School,Shop & Service,Shopping Mall,Spiritual Center,Travel & Transport,Vegetarian / Vegan Restaurant
0,8860145101fffff,1,"POLYGON ((77.51729 12.88649, 77.51737 12.89147...","Ganakkal, Hemmigepura, Rajarajeshwari Nagar Zo...",850.332284,4740.392847,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8860145105fffff,1,"POLYGON ((77.52162 12.89395, 77.52170 12.89894...","NICE Expressway, AGS Layout, Uttarahalli, Bomm...",2051.277533,4564.352886,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8860145107fffff,1,"POLYGON ((77.51320 12.89398, 77.51329 12.89897...","Ganakkal, Hemmigepura, Rajarajeshwari Nagar Zo...",850.332284,4449.431938,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,886014510dfffff,1,"POLYGON ((77.52571 12.88646, 77.52579 12.89144...","Banashankari 6th Stage - 2nd Block, Uttarahall...",2429.922136,4756.632313,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8860145111fffff,1,"POLYGON ((77.49613 12.87908, 77.49621 12.88407...","Hemmigepura, Hemmigepura Gollahalli, Bangalore...",850.332284,4281.226169,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [3]:
scaler = MaxAbsScaler()
feature_data = df.drop(columns = ['id', 'cluster', 'geometry', 'address', ])
scaled_features = scaler.fit_transform(feature_data)

df_features = pd.DataFrame(scaled_features, columns = feature_data.columns, index = df['id'])

df_features.columns

Index(['pop_density', 'cost_sqft', 'ATM', 'Arts & Entertainment',
       'Asian Restaurant', 'Athletics & Sports', 'Automotive Shop',
       'Bakery & Dessert', 'Bank', 'Cafeteria', 'Clothing & Jewelry',
       'Coffee & Tea', 'College & University', 'Electronics Store', 'Factory',
       'Fast Food', 'Gas Station', 'Groceries', 'Indian Restaurant',
       'Medical Center', 'Medical Store', 'Movie Theater', 'Nightlife Spot',
       'Office', 'Outdoors & Recreation', 'Professional & Other Places',
       'Quick Bites', 'Residence', 'Restaurant', 'Salon', 'School',
       'Shop & Service', 'Shopping Mall', 'Spiritual Center',
       'Travel & Transport', 'Vegetarian / Vegan Restaurant'],
      dtype='object')

We will use a weighted sum of selected features to create a score for each location. Complementary venues should increase the score and competition or other detractors should reduce the score.

A function is defined to calculate a location's score. An option has 

In [4]:
def scoreLocation(location, weights = None):
    # Equal weights for all features unless specified
    if not weights:
        cols = location.index
        n = len(cols)
        wt = [1/n for i in range(n)]
        weights = dict(zip(cols, wt))
        
    score = 0 # Initialize score
    for col, weight in weights.items():
        score += location[col] * weight
    
    return score

## Scenario 1: Casual Dining

For this type of restaurant, our primary competition is other similar restaurants, plus fast food or snack joints to some extent. Complementary venues would include residential areas, office locations, shopping malls and movie theaters - these should generally be good indicators of high footfall.

In [5]:
# Assign relative importance of different venues
weights = {
    'pop_density': 20,
    'cost_sqft': -20,
    'Asian Restaurant': -10,
    'Indian Restaurant': -7,
    'Restaurant': -7,
    'Vegetarian / Vegan Restaurant': -5,
    'Quick Bites': -3,
    'Fast Food': -3,
    'Residence': 10,
    'Office': 10,
    'Shopping Mall': 9,
    'Movie Theater': 9
}

In [6]:
scores = pd.Series(dtype = 'float')
counts = []
# For each cell, we will add the cells own score plus 20% of the score of neighbouring cells.
for id, row in df_features.iterrows():
    score = scoreLocation(row, weights)
    neighbours = list(h3.k_ring(id, 1))
    neighbours.remove(id) # Remove root cell - no double counting
    count = 0 # Number of neighbours (check)
    for n in neighbours:
        if n in df_features.index:
            score += (0.2 * scoreLocation(df_features.loc[n], weights))
            count +=1
    scores[id] = score
    counts.append(count)

df_scores_1 = df.copy().set_index('id', drop = False)
df_scores_1.insert(2, 'score', scores)
df_scores_1 = df_scores_1.sort_values('score', ascending = False)

# Display top 10 locations
pd.options.display.max_colwidth = 110
df_scores_1[['id', 'cluster', 'score', 'address']].head(10)

,id,cluster,score,address
id,,,,
8860145a21fffff,8860145a21fffff,1,2.596580,"Goripalya, Padarayanapura, West Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560104, India"
8861892eb7fffff,8861892eb7fffff,1,-1.158574,"Muneshwara Nagar, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560084, India"
8860145a2dfffff,8860145a2dfffff,1,-2.545365,"K H Ranganath Colony, Rayapuram Ward, West Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 5..."
8861892461fffff,8861892461fffff,3,-4.102479,"Mangammanapalya, Bommanahalli Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560068, India"
8860145b1dfffff,8860145b1dfffff,4,-4.575652,"Hospital-ayurvedic Homeo Clinic, 2nd Main Road, Prakash Nagar Ward, West Zone, Bengaluru, Bangalore North,..."
8861892eb5fffff,8861892eb5fffff,1,-5.065619,"Postal and Telecom Colony, Kadugondanahalli Ward, East Zone, Bengaluru, Bangalore North, Bangalore Urban, ..."
8860145b5bfffff,8860145b5bfffff,4,-5.230518,"Cheluvadipalya Road, Cottonpete, Chalavadipalya Ward, West Zone, Bengaluru, Bangalore North, Bangalore Urb..."
8860145a29fffff,8860145a29fffff,4,-5.470941,"Ashwathakatte Road, Kasturibai Nagar, Azad Nagar Ward, West Zone, Bengaluru, Bangalore North, Bangalore Ur..."
88618921b1fffff,88618921b1fffff,1,-5.815086,"Hudi, Mahadevapura Zone, K Dommasandra, Bangalore East, Bangalore Urban, Karnataka, 560067, India"


In [7]:
map_centre = (12.9792,77.5916)

map1 = folium.Map(location = map_centre, zoom_start = 11)

bins = [
    df_scores_1['score'].min(),
    df_scores_1['score'].quantile(0.50),
    df_scores_1['score'].quantile(0.85),
    df_scores_1['score'].quantile(0.95),
    df_scores_1['score'].quantile(0.99),
    df_scores_1['score'].max(),
]

choropleth = folium.Choropleth(
    geo_data = df_scores_1,
    data = df_scores_1['score'],
    key_on = 'id',
    fill_color = 'YlGnBu',
    fill_opacity = 0.8,
    nan_fill_opacity = 0.0,
    line_opacity = 0.9,
    legend_name = 'Score (higher is better)',
    bins = bins,
    highlight = True,
)

popup = folium.GeoJsonPopup(
    fields = ['id', 'address', 'score'],
    aliases = ['Hex ID', 'Address', 'Score'],
).add_to(choropleth.geojson)

map1.add_child(choropleth)

map1 # Display map

## Scenario 2: Pub

Here, our primary targets are college students and young working professionals - so look for areas near colleges or offices, that do not already have a lot of competition.

In [8]:
# Assign relative importance of different venues
weights = {
    'pop_density': 15,
    'cost_sqft': -25,
    'Nightlife Spot': -15.0,
    'Residence': 8,
    'Office': 8,
    'Shopping Mall': 10,
    'Movie Theater': 10,
    'College & University': 15,
    'Arts & Entertainment': 5
}

In [9]:
scores = pd.Series(dtype = 'float')
counts = []
# For each cell, we will add the cells own score plus 20% of the score of neighbouring cells.
for id, row in df_features.iterrows():
    score = scoreLocation(row, weights)
    neighbours = list(h3.k_ring(id, 1))
    neighbours.remove(id) # Remove root cell - no double counting
    count = 0 # Number of neighbours (check)
    for n in neighbours:
        if n in df_features.index:
            score += (0.2 * scoreLocation(df_features.loc[n], weights))
            count +=1
    scores[id] = score
    counts.append(count)

df_scores_2 = df.copy().set_index('id', drop = False)
df_scores_2.insert(2, 'score', scores)
df_scores_2 = df_scores_2.sort_values('score', ascending = False)

# Display top 10 locations
pd.options.display.max_colwidth = 110
df_scores_2[['id', 'cluster', 'score', 'address']].head(10)

,id,cluster,score,address
id,,,,
8861892185fffff,8861892185fffff,4,5.115337,"Kadugodi, Mahadevapura Zone, Sheegehalli, Bangalore East, Bangalore Urban, Karnataka, 56066, India"
88618925d9fffff,88618925d9fffff,0,-0.829186,"MICO Layout, BTM Layout Ward, South Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560069, ..."
8860169625fffff,8860169625fffff,1,-0.912819,"Chowdeswari Ward, Yelahanka Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560064, India"
88618925d5fffff,88618925d5fffff,4,-1.719818,"Bismillah Nagar, Gurappanapalya Ward, South Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, ..."
88618924edfffff,88618924edfffff,4,-2.030712,"Indian Institute of Management Bangalore, Bannerghatta Road, Arekere, Arakere, Bommanahalli Zone, Bengalur..."
8860145b1dfffff,8860145b1dfffff,4,-2.058699,"Hospital-ayurvedic Homeo Clinic, 2nd Main Road, Prakash Nagar Ward, West Zone, Bengaluru, Bangalore North,..."
88618921abfffff,88618921abfffff,3,-2.075682,"Hudi, Mahadevapura Zone, K Dommasandra, Bangalore East, Bangalore Urban, Karnataka, 56066, India"
8861892cd7fffff,8861892cd7fffff,3,-2.075933,"4th Cross Road, Anandanagar, Hebbala Ward, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnata..."
8861892c8bfffff,8861892c8bfffff,3,-2.261285,"Sulthanpalya, Vishwanath Nagenahalli, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 5..."


In [10]:
map_centre = (12.9792,77.5916)

map2 = folium.Map(location = map_centre, zoom_start = 11)

bins = [
    df_scores_2['score'].min(),
    df_scores_2['score'].quantile(0.50),
    df_scores_2['score'].quantile(0.85),
    df_scores_2['score'].quantile(0.95),
    df_scores_2['score'].quantile(0.99),
    df_scores_2['score'].max(),
]

choropleth = folium.Choropleth(
    geo_data = df_scores_2,
    data = df_scores_2['score'],
    key_on = 'id',
    fill_color = 'YlGnBu',
    fill_opacity = 0.8,
    nan_fill_opacity = 0.0,
    line_opacity = 0.9,
    legend_name = 'Score (higher is better)',
    bins = bins,
    highlight = True,
)

popup = folium.GeoJsonPopup(
    fields = ['id', 'address', 'score'],
    aliases = ['Hex ID', 'Address', 'Score'],
).add_to(choropleth.geojson)

map2.add_child(choropleth)

map2 # Display map